## Assignment 2

### Service Rate Control

Consider a discrete-time single-server queueing system that is observed
every $\eta > 0$ units of time. The controller makes decisions at times
$0, \eta, 2\eta, \dots$. Jobs arrive following a Poisson distribution
with rate 1.5 jobs per period of length $\eta$. There is a finite system
capacity of eight units; that is, if arriving jobs cause the system
content to exceed eight units, excess jobs do not enter the system and
are lost.

At each decision epoch, the controller observes the number of jobs in
the system and selects the service rate from a set of probability
distributions indexed by elements of the set $B = \{0, 1, 2\}$. For each
$b \in B$, let $f_b(n)$ denote the probability of $n$ service
completions within a period of length $\eta$ with:

-   $f_0(1) = 0.8$, $f_0(2) = 0.2$
-   $f_1(1) = 0.5$, $f_1(2) = 0.5$
-   $f_2(1) = 0.2$, $f_2(2) = 0.8$

The stationary reward structure consists of four components:

1.  A constant reward $R = 5$ for every completed service.
2.  An expected holding cost $h(s) = 2s$ per period when there are $s$
    jobs in the system.
3.  A fixed cost $K = 3$ for changing the service rate.
4.  A per-period cost $d(b)$ for using service rate $b$, where
    $d(0) = 0$, $d(1) = 2$, and $d(2) = 5$.

Determine a minimum-cost service rate control policy.

#### (a)

-   Formulate the problem above as an infinite horizon Markov decision
    problem.
-   Choose the optimality criterion that you find most reasonable
    (average costs or discounted costs). Also, choose a method (or
    methods) for computing the optimal policies and the value. Motivate
    your choices.
-   Develop the model and the algorithm. Compute the optimal policies
    and the value. *(Note: you should write your own code for the
    algorithm, i.e., do not use an existing MDP implementation that is
    available as a code library or on the internet).*

Please Report: - Model description - Your choice for an optimality
criterion including motivation - Solution algorithm (including
motivation) - Numerical results and a discussion of those

#### (b)

Now, we require that the server may work at service rate $b = 2$ at most
25% of the time. Model and solve this adjusted problem.

------------------------------------------------------------------------

## Solution

The transition from state $i$ to $j$ of the queuing system is as
follows:

-   After state $i$ the controller chooses a service rate

-   After jobs (0, 1 or 2) have departed the system

-   New jobs arrive

-   The current number of jobs in the system is state $j$

<h2>System State Timeline</h2>
<div id="sketch-holder"></div>


<script>
    function setup() {
        const canvas = createCanvas(800, 200);
        canvas.parent('sketch-holder');
        textAlign(CENTER, CENTER);
        textSize(14);
    }

    function draw() {
        background(255);
        
        // Timeline base
        const startX = 50;
        const endX = 750;
        const baseY = 100;
        strokeWeight(2);
        line(startX, baseY, endX, baseY);
        
        // Draw events
        const events = [
            { x: 150, label: "State i", symbol: "i" },
            { x: 300, label: "Choose\nservice rate", symbol: "μ" },
            { x: 450, label: "Jobs depart\n(0,1,2)", symbol: "↓" },
            { x: 600, label: "New jobs\narrive", symbol: "↑" },
            { x: 700, label: "State j", symbol: "j" }
        ];
        
        // Draw arrows and events
        strokeWeight(1);
        for (let i = 0; i < events.length; i++) {
            const event = events[i];
            
            // Draw vertical line
            line(event.x, baseY - 20, event.x, baseY + 20);
            
            // Draw label
            fill(0);
            noStroke();
            text(event.label, event.x, baseY + 45);
            
            // Draw state/symbol
            textSize(16);
            text(event.symbol, event.x, baseY - 35);
            textSize(14);
            
            // Draw arrow to next event if not last event
            if (i < events.length - 1) {
                stroke(0);
                drawArrow(event.x, baseY, events[i + 1].x - 40, baseY);
            }
        }
    }
    
    function drawArrow(x1, y1, x2, y2) {
        line(x1, y1, x2, y2);
        push();
        translate(x2, y2);
        const angle = atan2(y2 - y1, x2 - x1);
        rotate(angle);
        const arrowSize = 8;
        triangle(0, arrowSize/2, 0, -arrowSize/2, arrowSize, 0);
        pop();
    }
</script>

To formulate this problem as an infinite horizon Markov decision process
(MDP), we define the key components:

### State Space ($S$)

The state $s$ represents the number of jobs in the system, and it can
take values from $0$ to $8$ (since the system has a finite capacity of 8
units): $$ S = \{0, 1, 2, \dots, 8\} $$

### Action Space ($B$)

At each decision epoch, the controller can select a service rate
$b \in B$, where: $$ B = \{0, 1, 2\} $$

### Transition Probabilities ($P(j|i, b)$)

The transition probabilities depend on the arrival rate (Poisson
distribution with a rate of 1.5 jobs per period of length $\eta$) and
the service rate $b$ chosen.

-   Let $f_a(k)$ denote the probability of $k$ job arrivals in a period
    of length $\eta$. Since arrivals follow a Poisson distribution with
    rate 1.5:
    $$ f_a(k) = \frac{(1.5)^k e^{-1.5}}{k!}, \quad k = 0, 1, 2, \dots $$

-   Let $f_b(n)$ denote the probability of $n$ service completions under
    service rate $b$, where $f_b(1)$ and $f_b(2)$ are given for each
    $b \in B$.

Define $p(j | i, a)$ as the probability of transitioning to state $j$
when the system is in state $i$ and action $a$ is chosen. We have the
following cases:

-   $i = 0 \ or\  1$: the system is empty or contains only one job and
    the controler has to choose a service rate $b$. The choice does not
    affect $j$ because there are either no jobs in the system to be
    processed or the system will be empty with probability 1 after
    processing. The number of jobs in state $j$ equals the number of
    arrivals.
-   $1 < i \leq 8$: the system contains $i$ items and the controler has
    to choose a service rate $b$. The number of items in the system
    after processing depends on the number of items processed. The
    number of items in the system in state $j$ will be
    $min(i - d + k, 8)$ with probability:

$$
\begin{cases}
\sum_d{f_b(n) * f_a(s_j - s_i + n)} & \text{if } s_j < 8 \\
\sum_d{f_b(n) * (1-\sum_{k=0}^{s_i - n - 1}f_a(k))} & \text{if } s_j = 8 \\
\end{cases}
$$

### Cost Function ($C(s, b, s')$)

The cost function consists of four components:

1.  **Reward for service completions**: For each service completion $n$,
    the system receives a reward of $R = 5$. The expected number of
    service completions when the system is in state $s$ and action $b$
    is chosen is: $$ 5n $$

2.  **Holding cost**: The expected holding cost per period is
    $h(s) = 2s$ when there are $s$ jobs in the system.

3.  **Cost of changing service rate**: If the controller changes the
    service rate from one period to the next, a fixed cost of $K = 3$ is
    incurred.

4.  **Per-period cost for using service rate**: The cost $d(b)$ depends
    on the chosen service rate $b$, where:
    $$ d(0) = 0, \quad d(1) = 2, \quad d(2) = 5 $$

The total cost function can thus be written as:
$$ C(s, b, s') = h(s) + d(b) + K \cdot \mathbf{1}_{\{b \text{ changed}\}} - 5 n$$

### Objective

The objective is to find a service rate control policy $\pi(s)$ that
minimizes the total expected cost over an infinite horizon. This is
typically done by solving the Bellman equation:
$$ V(s) = \min_{b \in B} \left[ C(s, b, s') + \sum_{s'} P(s'|s, b) V(s') \right] $$
where $V(s)$ is the value function representing the minimum expected
cost starting from state $s$.

### Optimality criterion: discounted reward


In [1]:
from ortools.linear_solver import pywraplp
from scipy.stats import poisson

# Create the solver
solver = pywraplp.Solver.CreateSolver('GLOP')

# max system capacity
S_max = 8

# discount factor lambda
l = 0.95

# period cost d(b) for using server state b for one period
d = {
        0: 0,
        1: 2,
        2: 5
    }

# cost K for changing the service rate
K = 3

# reward R for processing 1 job per period
R = 5

# costs for using server state b_prime when previous 
# server state was b. These consist of a period cost of using 
# server state b_prime and a cost for changing server state if
# b_prime != b.
def c(b, b_prime):
    if b_prime == b:
        return d[b_prime]
    else:
        return d[b_prime] + 3

def h(s):
    return 2 * s

# probability f_b(n) of processing n jobs if the service rate is b
def f(n, b):
    b_prob = {0: [0.8, 0.2], 1: [0.5, 0.5], 2: [0.2, 0.8]}
    return b_prob[b][n - 1]

# probability g(n) of n new jobs arriving
def g(n):
    return poisson.pmf(n, mu=1.5)

# probability of n or more new jobs arriving
def g_or_more(n):
    return 1 - sum(g(m) for m in range(n))

# Sets of states (S: number of jobs in the system, B: server states)
S = range(S_max + 1)
B = [0, 1, 2]

# Define the expected reward R_{(s,b),a'} when state is (s,b) and chosen service rate is a'
def r(s, b, a):
    if s == 0:
        return - c(b, a)
    elif s == 1:
        return R * 1.0 - h(s) - c(b, a)
    else:
        return R * sum(f(n, a) * n for n in [1, 2]) - h(s) - c(b, a)

# Transition probabilities
def P(s_i, s_j, b_j, a):
    if b_j != a:
        return 0.0
    if s_i <= 1:
        if s_j < S_max:
            return g(s_j)
        elif s_j == S_max:
            return g_or_more(s_j)
        else:
            return 0.0
    elif s_i <= S_max:
        if s_j < S_max:
            return sum(f(n, b_j) * g(s_j - s_i + n) for n in [1, 2])
        elif s_j == S_max:
            return sum(f(n, b_j) * g_or_more(s_j - s_i + n) for n in [1, 2])
        else:
            return 0.0
    else:
        return 0.0

# Decision variables x_{(s,b),b'}: probability of choosing server state b' when state is (s,b)
x = {}
for s in S:
    for b in B:
        for b_prime in B:
            x[(s, b, b_prime)] = solver.NumVar(0.0, 1.0, f'x_{s}_{b}_{b_prime}')

# Initial state probabilities alpha
alpha = {}
n_total_states = len(S) * len(B)
for s in S:
    for b in B:
        alpha[(s, b)] = 1 / n_total_states
        
# Objective: maximize the expected reward    
objective = solver.Objective()
for s in S:
    for b in B:
        for a in B:
            objective.SetCoefficient(x[(s, b, a)], r(s, b, a))
objective.SetMaximization()

# Balance constraints
for s_j in S:
    for b_j in B:
        lhs = solver.Sum(x[(s_j, b_j, a)] for a in B)
        rhs = l * solver.Sum(
          P(s_i, s_j, b_j, a) * x[(s_i, b_i, a)] 
          for s_i in S for b_i in B for a in B
          )
        solver.Add(lhs == alpha[(s_j, b_j)] + rhs)

# Solve the problem
status = solver.Solve()

# Print the optimal solution
if status == pywraplp.Solver.OPTIMAL:
    for s in S:
        print(f"If {s} jobs in the system:")
        for b in B:
            print(f"   If previous service rate was {b}:")
            for a in B:
                x_opt = x[(s, b, a)].solution_value()
                if x_opt > 0:
                    print(f"      Use service rate {a} (x={x_opt})")
else:
    print("No feasible solution found.")

If 0 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.5515694772088197)
   If previous service rate was 1:
      Use service rate 1 (x=0.412440442871158)
   If previous service rate was 2:
      Use service rate 0 (x=0.18750922188314792)
If 1 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=1.0)
      Use service rate 1 (x=0.009833129370600473)
   If previous service rate was 1:
      Use service rate 1 (x=0.8271892020120805)
   If previous service rate was 2:
      Use service rate 0 (x=0.4882627887525087)
If 2 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=1.0)
      Use service rate 1 (x=0.04305622179645904)
   If previous service rate was 1:
      Use service rate 1 (x=1.0)
      Use service rate 2 (x=0.041286720618086234)
   If previous service rate was 2:
      Use service rate 2 (x=0.8460416296644189)
If 3 jobs in the system:
   If previous service rate was 0:
      Use serv

### Optimality criterion: average reward


In [2]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('GLOP')
solver.SetTimeLimit(60000)  # Set time limit in milliseconds (60 seconds)

# Parameters
d = {0: 0, 1: 2, 2: 5}  # Period cost for each server state
K = 3  # Cost for changing the service rate
R = 5  # Reward for processing 1 job per period
S_max = 8  # Max system capacity

# Cost function for using server state b_prime when previous server state was b
def c(b, b_prime):
    return d[b_prime] if b == b_prime else d[b_prime] + 3

def h(s):
    return 2 * s

# Probability f_b(n) of processing n jobs if the service rate is b
def f(n, b):
    if b == 0:
        return 0.8 if n == 1 else 0.2 if n == 2 else 0.0
    elif b == 1:
        return 0.5 if n in {1, 2} else 0.0
    elif b == 2:
        return 0.2 if n == 1 else 0.8 if n == 2 else 0.0
    return 0.0

# Probability g(n) of n new jobs arriving
def g(n):
    return poisson.pmf(n, mu=1.5)

# Probability of n or more new jobs arriving
def g_or_more(n):
    return 1 - sum(g(m) for m in range(n))

# Sets of states (S: number of jobs in the system, B: server states)
S = range(S_max + 1)
B = [0, 1, 2]

# Define the expected reward R_{(s,b),a'} when state is (s,b) and chosen service rate is a
def r(s, b, a):
    if s == 0:
        return -c(b, a)
    elif s == 1:
        return R - h(s) - c(b, a)
    else:
        return R * sum(f(n, a) * n for n in [1, 2]) - h(s) - c(b, a)

# Transition probabilities
def P(s_i, b_i, s_j, b_j, a):
    if b_j != a:
        return 0.0
    if s_i == 0:
        return g(s_j) if s_j < S_max else g_or_more(s_j)
    elif s_i == 1:
        return g(s_j) if s_j < S_max else g_or_more(s_j)
    elif s_i <= S_max:
        if s_j < S_max:
            return sum(f(n, b_j) * g(s_j - s_i + n) for n in [1, 2])
        elif s_j == S_max:
            return sum(f(n, b_j) * g_or_more(s_j - s_i + n) for n in [1, 2])
    return 0.0

# Decision variables x_{(s,b),b'}: probability of choosing server state b' when state is (s,b)
x = {}
for s in S:
    for b in B:
        for b_prime in B:
            x[(s, b, b_prime)] = solver.NumVar(0.0, 1.0, f'x_{s}_{b}_{b_prime}')

# Initial state probabilities alpha
alpha = {}
n_total_states = len(S) * len(B)
for s in S:
    for b in B:
        alpha[(s, b)] = 1 / n_total_states

# Objective: maximize the expected reward
objective = solver.Objective()
for s in S:
    for b in B:
        for a in B:
            objective.SetCoefficient(x[(s, b, a)], r(s, b, a))
objective.SetMaximization()

# Balance constraints
for s_j in S:
    for b_j in B:
        # Right-hand side (outflow) - the sum of probabilities of choosing each action a in state (s_j, b_j)
        outflow = solver.Sum(x[(s_j, b_j, a)] for a in B)
        
        # Left-hand side (inflow) - the weighted sum of transition probabilities for each incoming state-action pair
        inflow = solver.Sum(
            P(s_i, b_i, s_j, b_j, a) * x[(s_i, b_i, a)]
            for s_i in S for b_i in B for a in B
        )
        
        # Add the constraint that the inflow and outflow must be equal
        solver.Add(inflow == outflow)

# Probability constraint to ensure all probabilities sum to 1
solver.Add(solver.Sum(x[(s, b, a)] for s in S for b in B for a in B) == 1)

# Solve the problem
status = solver.Solve()

# Print the optimal solution
if status == pywraplp.Solver.OPTIMAL:
    for s in S:
        print(f"If {s} jobs in the system:")
        for b in B:
            print(f"   If previous service rate was {b}:")
            for a in B:
                x_opt = x[(s, b, a)].solution_value()
                if x_opt > 0:
                    print(f"      Use service rate {a} (x={x_opt:.2f})")
else:
    print("No feasible solution found.")

If 0 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.06)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 0 (x=0.02)
If 1 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.10)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 0 (x=0.06)
If 2 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.09)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 2 (x=0.10)
If 3 jobs in the system:
   If previous service rate was 0:
      Use service rate 2 (x=0.05)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 2 (x=0.12)
If 4 jobs in the system:
   If previous service rate was 0:
      Use service rate 2 (x=0.02)
   If previous service rate was 1:
   If previous service rate was 2:
      Use service rate 2 (x=0.11)
If 5 jobs 

### Optimality criterion: discounted reward with service rate restriction


In [3]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('GLOP')

# max system capacity
S_max = 8

# discount factor lambda
l = 0.95

# period cost d(b) for using server state b for one period
d = {
        0: 0,
        1: 2,
        2: 5
    }

# cost K for changing the service rate
K = 3

# reward R for processing 1 job per period
R = 5

# costs for using server state b_prime when previous 
# server state was b. These consist of a period cost of using 
# server state b_prime and a cost for changing server state if
# b_prime != b.
def c(b, b_prime):
    if b_prime == b:
        return d[b_prime]
    else:
        return d[b_prime] + 3

def h(s):
    return 2 * s

# probability f_b(n) of processing n jobs if the service rate is b
def f(n, b):
    b_prob = {0: [0.8, 0.2], 1: [0.5, 0.5], 2: [0.2, 0.8]}
    return b_prob[b][n - 1]

# probability g(n) of n new jobs arriving
def g(n):
    return poisson.pmf(n, mu=1.5)

# probability of n or more new jobs arriving
def g_or_more(n):
    return 1 - sum(g(m) for m in range(n))

# Sets of states (S: number of jobs in the system, B: server states)
S = range(S_max + 1)
B = [0, 1, 2]



# Define the expected reward R_{(s,b),a'} when state is (s,b) and chosen service rate is a'
def r(s, b, a):
    if s == 0:
        return 0.0 - c(b, a)
    elif s == 1:
        return R * 1.0 - h(s) - c(b, a)
    else:
        return R * sum(f(n, a) * n for n in [1, 2]) - h(s) - c(b, a)

# Transition probabilities
def P(s_i, s_j, b_j, a):
    if b_j != a:
        return 0.0
    if s_i <= 1:
        if s_j < S_max:
            return g(s_j)
        elif s_j == S_max:
            return g_or_more(s_j)
        else:
            return 0.0
    elif s_i <= S_max:
        if s_j < S_max:
            return sum(f(n, b_j) * g(s_j - s_i + n) for n in [1, 2])
        elif s_j == S_max:
            return sum(f(n, b_j) * g_or_more(s_j - s_i + n) for n in [1, 2])
        else:
            return 0.0
    else:
        return 0.0

# Decision variables x_{(s,b),b'}: probability of choosing server state b' when state is (s,b)
x = {}
for s in S:
    for b in B:
        for b_prime in B:
            x[(s, b, b_prime)] = solver.NumVar(0.0, 1.0, f'x_{s}_{b}_{b_prime}')

# Initial state probabilities alpha
alpha = {}
n_total_states = len(S) * len(B)
for s in S:
    for b in B:
        alpha[(s, b)] = 1 / n_total_states
        
# Objective: maximize the expected reward    
objective = solver.Objective()
for s in S:
    for b in B:
        for a in B:
            objective.SetCoefficient(x[(s, b, a)], r(s, b, a))
objective.SetMaximization()

# Balance constraints
for s_j in S:
    for b_j in B:
        lhs = solver.Sum(x[(s_j, b_j, a)] for a in B)
        rhs = l * solver.Sum(
          P(s_i, s_j, b_j, a) * x[(s_i, b_i, a)] 
          for s_i in S for b_i in B for a in B
          )
        solver.Add(lhs == alpha[(s_j, b_j)] + rhs)
        
# Define the left-hand side: the total probability of using server state b = 2
service_rate_b2 = solver.Sum(x[(s, 2, a)] for s in S for a in B)

# Define the right-hand side: 25% of the total probability across all states, server states, and actions
total_probability = solver.Sum(x[(s, b, a)] for s in S for b in B for a in B)
solver.Add(service_rate_b2 <= 0.25 * total_probability)

# Solve the problem
status = solver.Solve()

# Print the optimal solution
if status == pywraplp.Solver.OPTIMAL:
    for s in S:
        print(f"If {s} jobs in the system:")
        for b in B:
            print(f"   If previous service rate was {b}:")
            for a in B:
                x_opt = x[(s, b, a)].solution_value()
                if x_opt > 0:
                    print(f"      Use service rate {a} (x={x_opt})")
else:
    print("No feasible solution found.")

If 0 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.5391421528072634)
   If previous service rate was 1:
      Use service rate 0 (x=0.34956839328894035)
   If previous service rate was 2:
      Use service rate 0 (x=0.043287273129876985)
If 1 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=1.0)
   If previous service rate was 1:
      Use service rate 0 (x=0.05381167876673675)
      Use service rate 1 (x=0.7182834160071975)
   If previous service rate was 2:
      Use service rate 0 (x=0.18290524885797413)
If 2 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=1.0)
      Use service rate 1 (x=0.10515373112308939)
   If previous service rate was 1:
      Use service rate 1 (x=1.0)
   If previous service rate was 2:
      Use service rate 1 (x=0.4070549388669085)
      Use service rate 2 (x=0.03685738787409642)
If 3 jobs in the system:
   If previous service rate was 0:
      Use se

### Optimality criterion: average reward with service rate restriction


In [4]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('GLOP')
solver.SetTimeLimit(60000)  # Set time limit in milliseconds (60 seconds)

# Parameters
d = {0: 0, 1: 2, 2: 5}  # Period cost for each server state
K = 3  # Cost for changing the service rate
R = 5  # Reward for processing 1 job per period
S_max = 8  # Max system capacity

# Cost function for using server state b_prime when previous server state was b
def c(b, b_prime):
    return d[b_prime] if b == b_prime else d[b_prime] + 3

def h(s):
    return 2 * s

# Probability f_b(n) of processing n jobs if the service rate is b
def f(n, b):
    if b == 0:
        return 0.8 if n == 1 else 0.2 if n == 2 else 0.0
    elif b == 1:
        return 0.5 if n in {1, 2} else 0.0
    elif b == 2:
        return 0.2 if n == 1 else 0.8 if n == 2 else 0.0
    return 0.0

# Probability g(n) of n new jobs arriving
def g(n):
    return poisson.pmf(n, mu=1.5)

# Probability of n or more new jobs arriving
def g_or_more(n):
    return 1 - sum(g(m) for m in range(n))

# Sets of states (S: number of jobs in the system, B: server states)
S = range(S_max + 1)
B = [0, 1, 2]

# Define the expected reward R_{(s,b),a'} when state is (s,b) and chosen service rate is a
def r(s, b, a):
    if s == 0:
        return -c(b, a)
    elif s == 1:
        return R - h(s) - c(b, a)
    else:
        return R * sum(f(n, a) * n for n in [1, 2]) - h(s) - c(b, a)

# Transition probabilities
def P(s_i, b_i, s_j, b_j, a):
    if b_j != a:
        return 0.0
    if s_i == 0:
        return g(s_j) if s_j < S_max else g_or_more(s_j)
    elif s_i == 1:
        return g(s_j) if s_j < S_max else g_or_more(s_j)
    elif s_i <= S_max:
        if s_j < S_max:
            return sum(f(n, b_j) * g(s_j - s_i + n) for n in [1, 2])
        elif s_j == S_max:
            return sum(f(n, b_j) * g_or_more(s_j - s_i + n) for n in [1, 2])
    return 0.0

# Decision variables x_{(s,b),b'}: probability of choosing server state b' when state is (s,b)
x = {}
for s in S:
    for b in B:
        for b_prime in B:
            x[(s, b, b_prime)] = solver.NumVar(0.0, 1.0, f'x_{s}_{b}_{b_prime}')

# Initial state probabilities alpha
alpha = {}
n_total_states = len(S) * len(B)
for s in S:
    for b in B:
        alpha[(s, b)] = 1 / n_total_states

# Objective: maximize the expected reward
objective = solver.Objective()
for s in S:
    for b in B:
        for a in B:
            objective.SetCoefficient(x[(s, b, a)], r(s, b, a))
objective.SetMaximization()

# Balance constraints
for s_j in S:
    for b_j in B:
        # Right-hand side (outflow) - the sum of probabilities of choosing each action a in state (s_j, b_j)
        outflow = solver.Sum(x[(s_j, b_j, a)] for a in B)
        
        # Left-hand side (inflow) - the weighted sum of transition probabilities for each incoming state-action pair
        inflow = solver.Sum(
            P(s_i, b_i, s_j, b_j, a) * x[(s_i, b_i, a)]
            for s_i in S for b_i in B for a in B
        )
        
        # Add the constraint that the inflow and outflow must be equal
        solver.Add(inflow == outflow)

# Probability constraint to ensure all probabilities sum to 1
solver.Add(solver.Sum(x[(s, b, a)] for s in S for b in B for a in B) == 1)

# Service rate restriction: Limit the probability of using server state b = 2 to 25%
service_rate_limit = solver.Sum(x[(s, 2, a)] for s in S for a in B)
solver.Add(service_rate_limit <= 0.25)

# Solve the problem
status = solver.Solve()

# Print the optimal solution
if status == pywraplp.Solver.OPTIMAL:
    for s in S:
        print(f"If {s} jobs in the system:")
        for b in B:
            print(f"   If previous service rate was {b}:")
            for a in B:
                x_opt = x[(s, b, a)].solution_value()
                if x_opt > 0:
                    print(f"      Use service rate {a} (x={x_opt:.2f})")
else:
    print("No feasible solution found.")

If 0 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.00)
   If previous service rate was 1:
      Use service rate 1 (x=0.05)
   If previous service rate was 2:
If 1 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.01)
   If previous service rate was 1:
      Use service rate 1 (x=0.10)
   If previous service rate was 2:
      Use service rate 0 (x=0.01)
If 2 jobs in the system:
   If previous service rate was 0:
      Use service rate 0 (x=0.01)
   If previous service rate was 1:
      Use service rate 1 (x=0.12)
   If previous service rate was 2:
      Use service rate 1 (x=0.02)
If 3 jobs in the system:
   If previous service rate was 0:
      Use service rate 1 (x=0.00)
   If previous service rate was 1:
      Use service rate 1 (x=0.11)
   If previous service rate was 2:
      Use service rate 2 (x=0.03)
If 4 jobs in the system:
   If previous service rate was 0:
      Use service rate 2 (x=0.00)
   If previo